In [ ]:
#download all speech data from s3 before running inference
!aws s3 cp s3://monlam.ai.stt/tsv/latest.csv 04_combine_all.csv

In [1]:
!pip install transformers tqdm
# Install PyTorch (Choose the appropriate command for your setup from the PyTorch website)
!pip install torch torchvision torchaudio

# Or install TensorFlow
!pip install tensorflow


  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchvision-0.18.1-cp310-cp310-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached torchaudio-2.3.1-cp310-cp310-manylinux1_x86_64.whl.metadata (6.4 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 k

In [2]:
# Install PyTorch (Choose the appropriate command for your setup from the PyTorch website)
!pip install torch torchvision torchaudio

# Or install TensorFlow
!pip install tensorflow
!pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.8 MB/s eta 0:00:0000:01


In [ ]:
import pandas as pd 

df = pd.read_csv('04_combine_all.csv')

In [ ]:
df.__len__()

In [ ]:
df.head()

In [ ]:
df['path'] = df['url'].apply(lambda x: '/'.join(x.split('/')[-2:]))

In [ ]:
df['inf'] = " "

In [ ]:
df.head()

In [ ]:
df.to_csv('latest.csv')

In [ ]:
from tqdm.auto import tqdm
from transformers import pipeline
from pathlib import Path

generator = pipeline(model="spsither/mms_300_v3.1020")

predictions = []

for path in tqdm(df['path']):
    path = Path(path)
    if not path.exists(): 
        predictions.append('')
        continue
    opt = generator(str(path))
    predictions.append(opt["text"])

df['inf'] = predictions

In [ ]:
import pandas as pd
from tqdm.auto import tqdm
from transformers import pipeline
from pathlib import Path

In [ ]:
df = pd.read_csv('04_combine_all.csv')

In [ ]:
df.head()


In [ ]:
# Load the dataframe from the latest checkpoint
df = pd.read_csv("latest.csv")

# Initialize the pipeline
generator = pipeline(model="spsither/mms_300_v3.1020")

# Filter the dataframe to only include rows where the 'inf' column is empty
df_to_process = df[df['inf'] == " "]

# Counter to keep track of rows processed
counter = 0

# Run inference on the filtered dataframe
for index, path in tqdm(df_to_process['path'].items(), desc="Processing Paths", total=len(df_to_process)):
    path = Path(path)
    if not path.exists():
        df.at[index, 'inf'] = ' '
    else:
        opt = generator(str(path))
        df.at[index, 'inf'] = opt["text"]
    
    # Increment the counter
    counter += 1

    # Save the dataframe at every 100 intervals
    if counter % 100 == 0:
        df.to_csv("latest.csv", index=False)
        print(f"Saved progress at {counter} rows.")

# Save the final updated dataframe
df.to_csv("latest.csv", index=False)
print("Final save completed.")


In [ ]:
df.head(60)

In [ ]:
df.to_csv('04_combine_all.csv', index=False)

In [ ]:
# Define file paths and bucket details
# save combine with time stamp to s3 
local_file = '04_combine_all.csv'
bucket_name = 'monlam.ai.stt'
s3_file_name = 'tsv/04_combine_all_04_jun_2024.csv'

# Run the AWS CLI command to upload the file
!aws s3 cp {local_file} s3://{bucket_name}/{s3_file_name}

In [ ]:
import pandas as pd
import os

def split_csv(file_path, rows_per_file, output_dir):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Read the input CSV file
    df = pd.read_csv(file_path)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # Get the total number of rows in the DataFrame
    total_rows = len(df)

    # Calculate the number of files needed
    num_files = (total_rows // rows_per_file) + (1 if total_rows % rows_per_file != 0 else 0)

    # Split the DataFrame and save each part as a new CSV file
    for i in range(num_files):
        start_row = i * rows_per_file
        end_row = start_row + rows_per_file
        part_df = df.iloc[start_row:end_row]
        folder_dir = os.path.join(output_dir, f'SW{str(i+1).zfill(5)}')
        if not os.path.exists(folder_dir):
            os.makedirs(folder_dir)
        output_file = os.path.join(folder_dir, f'SW{str(i+1).zfill(5)}.csv')
        part_df = part_df.loc[:, ~part_df.columns.str.contains('^Unnamed')]
        part_df.to_csv(output_file, index=False)
    print(f'Successfully split {file_path} into {num_files} files.')


In [ ]:
# Specify the input file path and the number of rows per file
input_file_path = '04_combine_all.csv'
rows_per_file = 1000
output_dir = 'output'
# Split the CSV file
split_csv(input_file_path, rows_per_file, output_dir)


In [3]:
!pip install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 5.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 21.5 MB/s eta 0:00:0000:01


In [4]:
import os
from github import Github
import subprocess

def repo_exists(org, repo_name):
    """Check if a repository exists in the organization."""
    try:
        repo = org.get_repo(repo_name)
        return repo.clone_url
    except:
        return False

def create_github_repo(repo_name, org, token):
    # Authenticate to GitHub
    g = Github(token)
    
    # Get the organization
    org = g.get_organization(org)
    
    # Check if the repository already exists
    repo_url = repo_exists(org, repo_name)
    if repo_url:
        print(f'Repository {repo_name} already exists.')
        return repo_url
    
    # Create the repository
    try:
        repo = org.create_repo(name=repo_name)
        print(f'Successfully created repository: {repo_name}')
        return repo.clone_url
    except Exception as e:
        print(f'Failed to create repository: {repo_name}')
        print(f'Error: {e}')
        return None

def upload_files_to_repo(local_dir, repo_url, token):
    # Set Git configuration for user
    subprocess.run(['git', 'config', '--global', 'user.email', 'ganga@esukhia.org'])
    subprocess.run(['git', 'config', '--global', 'user.name', 'gangagyatso4364'])
    
    # Initialize a local git repository if it does not exist
    if not os.path.exists(os.path.join(local_dir, '.git')):
        subprocess.run(['git', 'init'], cwd=local_dir)
    
    # Add the remote origin, ignoring error if it already exists
    subprocess.run(['git', 'remote', 'remove', 'origin'], cwd=local_dir)
    subprocess.run(['git', 'remote', 'add', 'origin', repo_url], cwd=local_dir)
    
    # Pull from the remote repository to get the latest changes
    subprocess.run(['git', 'pull', 'origin', 'master'], cwd=local_dir)
    
    # Add all files to the staging area
    subprocess.run(['git', 'add', '.'], cwd=local_dir)
    
    # Commit the files
    subprocess.run(['git', 'commit', '-m', 'Initial commit'], cwd=local_dir)
    
    # Push the files to the GitHub repository
    subprocess.run(['git', 'push', 'origin', 'master'], cwd=local_dir)

def create_repos_for_folders(output_dir, org_name, token):
    for folder_name in os.listdir(output_dir):
        folder_path = os.path.join(output_dir, folder_name)
        if os.path.isdir(folder_path):
            repo_url = create_github_repo(folder_name, org_name, token)
            if repo_url:
                upload_files_to_repo(folder_path, repo_url.replace('https://', f'https://{token}@'), token)



In [ ]:
# Specify your GitHub organization name and personal access token
org_name = 'MonlamAI'
token = 'your_token'  # Replace with your actual token

# Output directory containing the folders
output_dir = 'output'

# Create repositories for each folder in the output directory
create_repos_for_folders(output_dir, org_name, token)


Repository SW00231 already exists.
Already up to date.
On branch master
nothing to commit, working tree clean


From https://github.com/MonlamAI/SW00231
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
Everything up-to-date


Repository SW00085 already exists.
Already up to date.
On branch master
nothing to commit, working tree clean


From https://github.com/MonlamAI/SW00085
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
Everything up-to-date


Repository SW00247 already exists.
Already up to date.
On branch master
nothing to commit, working tree clean


...

To https://github.com/MonlamAI/SW00504.git
 * [new branch]      master -> master
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/r

Failed to create repository: SW00381
Error: HTTPSConnectionPool(host='api.github.com', port=443): Max retries exceeded with url: /orgs/MonlamAI/repos (Caused by ResponseError('too many 403 error responses'))


Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden


Failed to create repository: SW00057
Error: HTTPSConnectionPool(host='api.github.com', port=443): Max retries exceeded with url: /orgs/MonlamAI/repos (Caused by ResponseError('too many 403 error responses'))


Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
Request POST /orgs/MonlamAI/repos failed with 403: Forbidden
Setting next backoff to 60s
